# Análise Exploratória de Dados (EDA)
## Preditor de Preços Imobiliários Regionais
### Projeto de Inteligência Artificial

---

**Fontes de Dados**:
- API Banco Central: IPCA, SELIC, PIB
- API IBGE: Censo, PIB Municipal, População
- Brasil API: Dados complementares

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print('✅ Bibliotecas importadas com sucesso!')

## 1. Carregamento dos Dados

In [ ]:
df_ml = pd.read_csv('../data/processed/dataset_ml.csv')
df_indicadores = pd.read_csv('../data/raw/indicadores_economicos.csv')

print(f"📊 Dataset ML: {df_ml.shape}")
print(f"📈 Indicadores Econômicos: {df_indicadores.shape}")

df_ml.head()

## 2. Análise Descritiva

In [ ]:
df_ml.describe()

In [ ]:
df_ml.info()

In [ ]:
missing = df_ml.isnull().sum()
missing_pct = (missing / len(df_ml)) * 100
missing_df = pd.DataFrame({
    'Valores Faltantes': missing,
    'Percentual (%)': missing_pct
}).sort_values('Valores Faltantes', ascending=False)

print(missing_df[missing_df['Valores Faltantes'] > 0])

## 3. Análise da Variável Target

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df_ml['valorizacao_esperada_12m'], bins=50, color='skyblue', edgecolor='black')
axes[0].axvline(df_ml['valorizacao_esperada_12m'].mean(), color='red', linestyle='--', label='Média')
axes[0].axvline(df_ml['valorizacao_esperada_12m'].median(), color='green', linestyle='--', label='Mediana')
axes[0].set_xlabel('Valorização Esperada (%)')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição da Valorização Esperada (12 meses)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(df_ml['valorizacao_esperada_12m'], vert=True)
axes[1].set_ylabel('Valorização Esperada (%)')
axes[1].set_title('Box Plot da Valorização')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Média: {df_ml['valorizacao_esperada_12m'].mean():.2f}%")
print(f"Mediana: {df_ml['valorizacao_esperada_12m'].median():.2f}%")
print(f"Desvio Padrão: {df_ml['valorizacao_esperada_12m'].std():.2f}%")

## 4. Análise por Região

In [ ]:
valorizacao_regiao = df_ml.groupby('regiao')['valorizacao_esperada_12m'].agg(['mean', 'median', 'std', 'count'])
valorizacao_regiao.columns = ['Média', 'Mediana', 'Desvio Padrão', 'Nº Municípios']
valorizacao_regiao = valorizacao_regiao.sort_values('Média', ascending=False)

print("📊 Valorização por Região:")
print(valorizacao_regiao)

In [ ]:
fig = px.bar(
    valorizacao_regiao.reset_index(),
    x='regiao',
    y='Média',
    color='Média',
    color_continuous_scale='Greens',
    title='Valorização Média Esperada por Região',
    labels={'regiao': 'Região', 'Média': 'Valorização Média (%)'},
    text='Média'
)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.show()

## 5. Análise de Correlação

In [ ]:
vars_numericas = ['valorizacao_esperada_12m', 'pib_per_capita', 'populacao', 
                  'score_economico', 'ipca', 'selic']
vars_disponiveis = [v for v in vars_numericas if v in df_ml.columns]

correlation_matrix = df_ml[vars_disponiveis].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação - Variáveis Principais', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Análise de Indicadores Econômicos

In [ ]:
df_indicadores['data'] = pd.to_datetime(df_indicadores['data'])
df_indicadores = df_indicadores.sort_values('data')

df_temp = df_indicadores.tail(36)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

if 'ipca' in df_temp.columns:
    axes[0, 0].plot(df_temp['data'], df_temp['ipca'], marker='o', linewidth=2)
    axes[0, 0].set_title('IPCA (%)', fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)

if 'selic' in df_temp.columns:
    axes[0, 1].plot(df_temp['data'], df_temp['selic'], marker='o', linewidth=2, color='orange')
    axes[0, 1].set_title('Taxa SELIC (%)', fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)

if 'cambio' in df_temp.columns:
    axes[1, 0].plot(df_temp['data'], df_temp['cambio'], marker='o', linewidth=2, color='green')
    axes[1, 0].set_title('Taxa de Câmbio (R$/USD)', fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)

if 'desemprego' in df_temp.columns:
    axes[1, 1].plot(df_temp['data'], df_temp['desemprego'], marker='o', linewidth=2, color='red')
    axes[1, 1].set_title('Taxa de Desemprego (%)', fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Top Municípios Promissores

In [ ]:
top_12 = df_ml.nlargest(12, 'valorizacao_esperada_12m')[[
    'nome_municipio', 'uf', 'regiao', 'valorizacao_esperada_12m',
    'pib_per_capita', 'populacao', 'score_economico'
]].reset_index(drop=True)

top_12.index = top_12.index + 1
top_12.index.name = 'Ranking'

print("🏆 TOP 12 MUNICÍPIOS COM MAIOR POTENCIAL DE VALORIZAÇÃO")
print("=" * 80)
print(top_12.to_string())

## 8. Análise Bivariada

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

if 'pib_per_capita' in df_ml.columns:
    axes[0, 0].scatter(df_ml['pib_per_capita'], df_ml['valorizacao_esperada_12m'], alpha=0.5)
    axes[0, 0].set_xlabel('PIB per Capita (R$)')
    axes[0, 0].set_ylabel('Valorização Esperada (%)')
    axes[0, 0].set_title('Valorização vs PIB per Capita')
    axes[0, 0].grid(True, alpha=0.3)

if 'populacao' in df_ml.columns:
    axes[0, 1].scatter(df_ml['populacao'], df_ml['valorizacao_esperada_12m'], alpha=0.5, color='orange')
    axes[0, 1].set_xlabel('População')
    axes[0, 1].set_ylabel('Valorização Esperada (%)')
    axes[0, 1].set_xscale('log')
    axes[0, 1].set_title('Valorização vs População (escala log)')
    axes[0, 1].grid(True, alpha=0.3)

if 'score_economico' in df_ml.columns:
    axes[1, 0].scatter(df_ml['score_economico'], df_ml['valorizacao_esperada_12m'], alpha=0.5, color='green')
    axes[1, 0].set_xlabel('Score Econômico')
    axes[1, 0].set_ylabel('Valorização Esperada (%)')
    axes[1, 0].set_title('Valorização vs Score Econômico')
    axes[1, 0].grid(True, alpha=0.3)

if 'selic' in df_ml.columns:
    axes[1, 1].scatter(df_ml['selic'], df_ml['valorizacao_esperada_12m'], alpha=0.5, color='red')
    axes[1, 1].set_xlabel('Taxa SELIC (%)')
    axes[1, 1].set_ylabel('Valorização Esperada (%)')
    axes[1, 1].set_title('Valorização vs SELIC')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Insights e Conclusões

In [ ]:
print("💡 PRINCIPAIS INSIGHTS DA ANÁLISE EXPLORATÓRIA")
print("=" * 80)
print()

val_media = df_ml['valorizacao_esperada_12m'].mean()
print(f"1. Valorização média esperada: {val_media:.2f}% em 12 meses")
print()

regioes_altas = df_ml[df_ml['valorizacao_esperada_12m'] >= 15]
print(f"2. {len(regioes_altas)} municípios ({len(regioes_altas)/len(df_ml)*100:.1f}%) apresentam")
print(f"   valorização esperada ≥ 15%")
print()

if 'pib_per_capita' in vars_disponiveis:
    corr_pib = correlation_matrix.loc['valorizacao_esperada_12m', 'pib_per_capita']
    print(f"3. Correlação PIB per capita vs Valorização: {corr_pib:.3f}")
    if corr_pib > 0.3:
        print("   → Correlação positiva moderada/forte")
    print()

melhor_regiao = valorizacao_regiao.index[0]
val_melhor = valorizacao_regiao.iloc[0]['Média']
print(f"4. Região com maior potencial: {melhor_regiao}")
print(f"   Valorização média: {val_melhor:.2f}%")
print()

std_val = df_ml['valorizacao_esperada_12m'].std()
print(f"5. Desvio padrão da valorização: {std_val:.2f}%")
print("   → Indica" + (" alta" if std_val > 5 else " moderada") + " variabilidade entre municípios")
print()

print("=" * 80)